In [1]:
# importing libraries 

import numpy as np
import math

import scipy.special 

In [2]:
class NN:
    
    def __init__(self,input_nodes,hidden_nodes,output_classes,learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_classes = output_classes
        self.learning_rate = learning_rate
        
        # initializing weights randomly between input and hidden layers 
        self.w_ih = np.random.normal(loc = 0.0, scale = math.sqrt(1/self.hidden_nodes), size = (self.hidden_nodes,self.input_nodes))
        
        
        # activation function
        self.activation_function = lambda x: scipy.special.expit(x)        
        
        # initializing weights between hidden and output layers
        self.w_ho = np.random.normal(loc = 0.0, scale = math.sqrt(1/self.output_classes),size = (self.output_classes,self.hidden_nodes))
        
        pass
        
    def input_array(self,input_array): # for test data 
        # takes in 1D array. or one row of the data to predict output array([a,b,c,d,e,f,g])
        input_data = np.array(input_array,ndmin=2).T
        
        hidden_inputs = np.dot(self.w_ih,input_data) # dot product h = W.x
        
        # passing through activation function 
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # layer 2 
        
        class_input = np.dot(self.w_ho,hidden_outputs)
        class_outputs = self.activation_function(class_input)
        
        return class_outputs
    
        pass
    
    def train_data(self, input_array,target_array):
        input_data = np.array(input_array,ndmin=2).T
        target = np.array(target_array,ndmin=2).T
        
        # layer 1 
        hidden_inputs = np.dot(self.w_ih,input_data)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # layer 2 / output layer in this case 
        class_input = np.dot(self.w_ho,hidden_outputs)
        class_output = self.activation_function(class_input)
        
        # error calculation
        output_error = target - class_output
        
        hidden_error = np.dot(self.w_ho.T,output_error)
        
        # weight updating online learning 
        self.w_ho += self.learning_rate * np.dot(output_error*class_output*(1-class_output),np.transpose(hidden_outputs))
        self.w_ih += self.learning_rate * np.dot(hidden_error*hidden_outputs*(1-hidden_outputs),np.transpose(input_data))
        
        pass
        
        

In [3]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_classes = 10
# learning rate is 0.3
learning_rate = 0.1
# create instance of neural network
n = NN(input_nodes,hidden_nodes,output_classes,learning_rate)

In [4]:
training_data_file = open("mnist_train_100.csv","r")
train_data_list = training_data_file.readlines()
training_data_file.close()

In [5]:
len(train_data_list)

100

In [6]:
epochs = 500

for e in range(epochs):
    for line in train_data_list:
        value = line.split(",")

        # scale and shift 
        inputs = (np.asfarray(value[1:])/255 * 0.99) + 0.01

        # target labeling 
        targets = np.zeros(output_classes)+0.01

        targets[int(value[0])]  += 0.99

        n.train_data(inputs,targets)

        pass
    pass

In [7]:
# test data 
test_data_file = open("mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [8]:
# for accuracy 

pred = []

for data in test_data_list:
    val = data.split(",")
    
    right_label = int(val[0])
    
    test_input = (np.asfarray(val[1:])/255 *0.99) +0.01
    
    output = n.input_array(test_input)
    
    label = np.argmax(output)
    
    if (label==right_label):
        pred.append(1)
    else:
        pred.append(0)
        pass
    pass



In [9]:
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(pred)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.7
